# Notebook for TMC SaaS migration TOI demo

In [ ]:
uname -a
echo "PWD=$PWD"

## Clean up before execution

In [ ]:
namesuffix=$(date +%s)

tar -zcf data_$namesuffix.tar.gz ./data
ls -l data_$namesuffix.tar.gz

rm -rf ./data
ls -l ./data

## Connect to SaaS stack

Export the necessary environment variables to initialize the CLI context and exchange the API access token.

In [ ]:
export TANZU_API_TOKEN=g69o-MNlS6aUrMPK0PZlAACbcEApcLQHzn9UXl86TuyAaV9vRRG-M8lNnN_ptBSW
export ORG_NAME=trh
export TANZU_CLI_CEIP_OPT_IN_PROMPT_ANSWER=no
export TOKEN_URL=https://console-stg.tanzu.broadcom.com/csp/gateway/am/api/auth/api-tokens/authorize
export REFRESH_TOKEN=$TANZU_API_TOKEN

# Export filters
export CLUSTER_NAME_FILTER="wc-02"
export TMC_MC_FILTER="sc-803"


echo "ENV vars exported!"

Override the variables in STG environment, it's not required for customers.

In [ ]:
export CSP_URL=https://console-stg.tanzu.broadcom.com/csp/gateway/am/api/auth/api-tokens/authorize
export TMC_ENDPOINT=${ORG_NAME}.tmc-dev.tanzu.broadcom.com

Run **./001-base-saas_stack-connect.sh** to initialize the CLI context and exchange the API access token.

In [ ]:
echo "Create Tanzu CLI context and exchange an API access token for directly calling the API endpoint with curl."

echo "Environment variables needed:"

echo "TANZU_API_TOKEN=$TANZU_API_TOKEN"
echo "ORG_NAME=$ORG_NAME"
echo "TANZU_CLI_CEIP_OPT_IN_PROMPT_ANSWER=$TANZU_CLI_CEIP_OPT_IN_PROMPT_ANSWER"
echo "TOKEN_URL=$TOKEN_URL"
echo "REFRESH_TOKEN=$REFRESH_TOKEN"

tanzu config eula accept
./001-base-saas_stack-connect.sh

## Export the base resources

### Export cluster group

In [ ]:
# Export Cluster Groups
./002-base-clustergroups-export.sh

### Export workspaces

In [ ]:
# Export Workspaces
./003-base-workspaces-export.sh 

## Export administration resources

In [ ]:
# Export Roles
./004-admin-roles-export.sh

In [ ]:
# Export Credentials
./005-admin-credentials-export.sh

In [ ]:
# Export Access
./006-admin-access-export.sh

In [ ]:
# Export Proxy
./007-admin-proxy-export.sh

In [ ]:
# Export Image Registry
./008-admin-image-registry-export.sh

In [ ]:
# Export Settings
./009-admin-settings-export.sh

## Export add-on resources of cluster group

In [ ]:
# Export k8s secret resources of cluster groups
./010-clustergroup-secrets-export.sh

In [ ]:
# Export k8s secret export resources of cluster groups
./011-clustergroup-secret-exports-export.sh

In [ ]:
# Export fluxcd resources of cluster groups
./012-clustergroup-continuous-deliveries-export.sh

In [ ]:
# Export git repo credential resources of cluster groups
./013-clustergroup-repository-credentials-export.sh

In [ ]:
# Export git repository resources of cluster groups
./014-clustergroup-git-repositories-export.sh

In [ ]:
# Export kustomization resources of cluster groups
./015-clustergroup-kustomizations-export.sh

In [ ]:
# Export helm resources of cluster groups
./016-clustergroup-helms-export.sh

In [ ]:
# Export helm release resources of cluster groups
./017-clustergroup-helm-releases-export.sh

## Export add-on resources of clusters

In [ ]:
# Export managed namespace resources of clusters
./018-cluster-namespaces-export.sh

In [ ]:
# Export k8s secret resources of clusters
./019-cluster-secrets-export.sh

In [ ]:
# Export k8s secret export resources of clusters
./020-cluster-secret-exports-export.sh

In [ ]:
# Export fluxcd resources of clusters
./021-cluster-continuous-deliveries-export.sh

In [ ]:
# Export git repo credential resources of clusters
./022-cluster-repository-credentials-export.sh

In [ ]:
# Export git repository resources of clusters
./023-cluster-git-repositories-export.sh

In [ ]:
# Export kustomization resources of clusters
./024-cluster-kustomizations-export.sh

In [ ]:
# Export helm resources of clusters
./025-cluster-helms-export.sh

In [ ]:
# Export helm release resources of clusters
./026-cluster-helm-releases-export.sh

## Export data protection resources

In [ ]:
# Export data-protection
./027-cluster-data_protection-export.sh

## Export Polices

### ES-28: Export Access Policies from SaaS

In [ ]:
./028-base-access-policies-export.sh

### ES-29: Export Policy Templates

In [ ]:
./029-base-policy-templates-export.sh

### ES-30: Export Policy Assignments

In [ ]:
./030-base-policy-assignments-export.sh

## Offboard clusters from SaaS

### Managed clusters (VKS/TKG)

#### Export managed clusters
**NOTE:** Set environment variable `TMC_MC_FILTER="mc_cluster1,mc_cluster2"` to export the clusters under the specified management clusters only.

In [ ]:
export TMC_MC_FILTER="sc-803"
echo "TMC_MC_FILTER=$TMC_MC_FILTER"

./031-base-managed_clusters-export.sh

#### Offboard managed clusters

In [ ]:
./031-base-managed_clusters-offboard.sh

### Attached clusters

#### Export attached clusters

**NOTE:** Set environment variable `CLUSTER_NAME_FILTER="cluster1,cluster2"` to export the specified attached clusters only.

In [ ]:
export CLUSTER_NAME_FILTER="wc-02"
echo "CLUSTER_NAME_FILTER=$CLUSTER_NAME_FILTER"

./032-base-attached_clusters-export.sh

#### Offboard attached clusters

In [ ]:
./032-base-attached_clusters-offboard.sh

**NOTE: Cluster unmanage/detach/deregister may need a bit more time, even if the command is returned. Suggest double-checking from the SaaS stack to ensure all the clusters are offboarded before proceeding to the next steps.**

## Connect to the TMC SM stack to onboard clusters and restore resources

**NOTE:** Set the necessary environment variables to initialize the CLI context

**NOTE:** If the MFA feature of the IDP is enabled, export an extra environment variable.

`export TMC_SM_IDP_MFA_ENABLED=true`

In [ ]:
echo "Export environment variables for connecting SM stack"

export TMC_SELF_MANAGED_USERNAME=testuser01@tmcselfmanaged.com
export TMC_SELF_MANAGED_PASSWORD=VMware1!
export TMC_SELF_MANAGED_DNS=tmc.tanzu.io
export TMC_SM_CONTEXT=tmc-sm

echo "TMC_SELF_MANAGED_USERNAME=$TMC_SELF_MANAGED_USERNAME"
echo "TMC_SELF_MANAGED_PASSWORD=$TMC_SELF_MANAGED_PASSWORD"
echo "TMC_SELF_MANAGED_DNS=$TMC_SELF_MANAGED_DNS"
echo "TMC_SM_CONTEXT=$TMC_SM_CONTEXT"

echo "Run 033-base-sm_stack-connect.sh to init CLI context for SM stack"
./033-base-sm_stack-connect.sh

In [ ]:
tanzu context current

## Import resources PRE cluster onboard

### Import base resources

In [ ]:
# Import Cluster Groups
./034-base-clustergroups-import.sh

In [ ]:
# Import Workspaces
./035-base-workspaces-import.sh

### Import administration resources

In [ ]:
# Import Roles
./036-admin-roles-import.sh

In [ ]:
# Generate template yaml files for each credential And User need to fill in the missing values such as Crendentials and CA
./037-admin-credentials-create-template.sh

In [ ]:
# Import credentials
./037-admin-credentials-import.sh

In [ ]:
# Generate template yaml files for each proxy And User need to fill in the missing values such as Crendentials and CA
./038-admin-proxy-create-template.sh

In [ ]:
# Import Proxy Configuration
./038-admin-proxy-import.sh

In [ ]:
# Generate template yaml files for each Image-Registry And User need to fill in the missing values such as Crendentials and CA
./039-admin-image-registry-create-template.sh

In [ ]:
# Import Image-Registry
./039-admin-image-registry-import.sh

### Import add-on resources of cluster groups

In [ ]:
# Import k8s secret resources to cluster groups
./040-clustergroup-secrets-import.sh

In [ ]:
# Import k8s secret export resources to cluster groups
./041-clustergroup-secret-exports-import.sh

**USER ACTION REQUIRED TO IMPORT K8S SECRETS TO CLUSTER GROUPS**

Next step is importing k8s secret resource to the cluster groups

User must manually fill in the missing data fields depending on the type of k8s secret into the manifests in directory `./data/clustergroup-secrets`

* SECRET_TYPE_OPAQUE
  ```yaml
  spec:
    atomicSpec:
      data: # filled data field
        key1: base64-encoded-value1
        key2: base64-encoded-value2
      secretType: SECRET_TYPE_OPAQUE
  ```
* SECRET_TYPE_DOCKERCONFIGJSON
  ```yaml
  spec:
    atomicSpec:
      data: # filled data field
        .dockerconfigjson: base64-encoded-dockerconfig-json-file
      secretType: SECRET_TYPE_DOCKERCONFIGJSON
  ```

In [ ]:
# Import fluxcd resources to cluster groups
./042-clustergroup-continuous-deliveries-import.sh

In [ ]:
# Import git repository credential resources to cluster groups
./043-clustergroup-repository-credentials-import.sh

**USER ACTION REQUIRED TO IMPORT REPOSITORY CREDENTIALS TO CLUSTER GROUPS**

Users must manually fill in the missing data field depending on the type of credential into the manifests in directory `./data/clustergroup-repository-credentials`

* Username/Password
  ```yaml
  spec:
    atomicSpec:
      data: # filled data field
        data:
          username: bas64-encoded-username
          password: base64-encoded-password
      sourceSecretType: USERNAME_PASSWORD
  ```
* SSH Authentication
  ```yaml
  spec:
    atomicSpec:
      data: # filled data field
        data:
          identity: bas64-encoded-ssh-identity
          known_hosts: base64-encoded-ssh-known-hosts
      sourceSecretType: SSH
  ```
* CA Certificate
  ```yaml
  spec:
    atomicSpec:
      data: # filled data field
        data:
          ca.crt: bas64-encoded-ca-crt
          username: base64-encoded-username  # username and password are optional
          password: base64-encoded-password  # username and password are optional
      sourceSecretType: CACert
  ```

In [ ]:
# Import git repository resources to cluster groups
./044-clustergroup-git-repositories-import.sh

In [ ]:
# Import kustomization resources to cluster groups
./045-clustergroup-kustomizations-import.sh

In [ ]:
# Import helm resources to cluster groups
./046-clustergroup-helms-import.sh

In [ ]:
# Import helm release resources to cluster groups
./047-clustergroup-helm-releases-import.sh

## Onboard the exported clusters to the SM stack

### Managed clusters (VKS/TKGM)

Run script `048-base-managed_clusters-input_from_user.sh` to generate a placeholder kubeconfig index file of exported management clusters.

**NOTE**: Update the generated kubeconfig index file `clusters/mc-kubeconfig-index-file` by replacing the placeholder text '/path/to/the/real/mc_kubeconfig/file' with the real kubeconfig file of the onboarding management clusters before moving to the next step.


In [ ]:
./048-base-managed_clusters-input_from_user.sh
cat data/clusters/mc-kubeconfig-index-file

**NOTE:** If the onboarding clusters are exported without proxy settings, but proxy settings are needed for SM, modify the cluster configurations under `data/clusters/wc_of_{{MANAGEMENT-CLUSTER-NAME}}.yaml`.
If the onboarding clusters are exported with proxy settings, they will be configured with the same proxy settings that have been imported in the previous step.

Modify:
```yaml
spec:
  proxyName: <YOUR-PROXY-ON-SM>
```

**Run managed cluster onboard script**

In [ ]:
./048-base-managed_clusters-onboard.sh

### Attached clusters

Run script `049-base-attached_clusters-input_from_user.sh` to generate a placeholder kubeconfig index file of exported attached clusters

**NOTE**: Update the generated kubeconfig index file `clusters/attached-wc-kubeconfig-index-file` by replacing the placeholder text '/path/to/the/real/wc_kubeconfig/file' with the real kubeconfig file of the onboarding attached clusters before moving to the next step.

In [ ]:
./049-base-attached_clusters-input_from_user.sh
cat data/clusters/attached-wc-kubeconfig-index-file

**NOTE:** If the onboarding clusters are exported without proxy settings, but proxy settings are needed for SM, modify the cluster configurations under `clusters/attached_clusters.yaml`.
If the onboarding clusters are exported with proxy settings, they will be configured with the same proxy settings that have been imported in the previous step.

Modify:
```yaml
spec:
  proxyName: <YOUR-PROXY-ON-SM>
```

**Run the attached cluster onboard script**

In [ ]:
./049-base-attached_clusters-onboard.sh

### Check readiness of all onboarded clusters
Run the script to verify the readiness of all onboarded clusters to ensure they are healthy before beginning resource import.

In [ ]:
./049-base-whole_clusters-check_readiness.sh

## Import resources POST cluster onboard

### Import add-on resources of clusters

In [ ]:
# Import managed namespace resources to clusters
./050-cluster-namespaces-import.sh

In [ ]:
# Import k8s secret resources to clusters
./051-cluster-secrets-import.sh

**USER ACTION REQUIRED TO IMPORT K8S SECRETS TO CLUSTERS**

Users must manually fill in the missing data field depending on the type of k8s secret into the manifests in directory `./data/cluster-secrets`

* SECRET_TYPE_OPAQUE
  ```yaml
  spec:
    data: # filled data field
      your-secret-key1: base64-encoded-value1
      your-secret-key2: base64-encoded-value2
    secretType: SECRET_TYPE_OPAQUE
  ```
* SECRET_TYPE_DOCKERCONFIGJSON
  ```yaml
  spec:
    data: # filled data field
      .dockerconfigjson: base64-encoded-dockerconfig-json-file
    secretType: SECRET_TYPE_DOCKERCONFIGJSON
  ```

In [ ]:
# Import k8s secret export resources to clusters
./052-cluster-secret-exports-import.sh

In [ ]:
# Import fluxcd resources to clusters
./053-cluster-continuous-deliveries-import.sh

In [ ]:
# Import git repository credential resources to clusters
./054-cluster-repository-credentials-import.sh

**USER ACTION REQUIRED TO IMPORT REPOSITORY CREDENTIALS TO CLUSTERS**

Users must manually fill in the missing data field depending on the type of credential into the manifests in directory `./data/cluster-repository-credentials`

* Username/Password
  ```yaml
  spec:
    data: # filled data field
      data:
        username: bas64-encoded-username
        password: base64-encoded-password
    sourceSecretType: USERNAME_PASSWORD
  ```
* SSH Authentication
  ```yaml
  spec:
    data: # filled data field
      data:
        identity: bas64-encoded-ssh-identity
        known_hosts: base64-encoded-ssh-known-hosts
    sourceSecretType: SSH
  ```
* CA Certificate
  ```yaml
  spec:
    data: # filled data field
      data:
        ca.crt: bas64-encoded-ca-crt
        username: base64-encoded-username  # username and password are optional
        password: base64-encoded-password  # username and password are optional
    sourceSecretType: CACert
  ```

In [ ]:
# Import git repository resources to clusters
./055-cluster-git-repositories-import.sh

In [ ]:
# Import kustomization resources to clusters
./056-cluster-kustomizations-import.sh

In [ ]:
# Import helm resources to clusters
./057-cluster-helms-import.sh

In [ ]:
# Import helm releases resources to clusters
./058-cluster-helm-releases-import.sh

### Import administration resources

In [ ]:
# Import Admin Settings
./059-admin-settings-import.sh

In [ ]:
# Import Admin Access
./060-admin-access-import.sh

### Import Policies

#### ES-61: Import Access Policies
**Note**: You should edit the user and usergroup identities according to the idP in TMC SM after imported access policies.

* Import Access Policies on Organization, Clustergroups, Workspaces

In [ ]:
./061-base-access-policies-import.sh

* Import Access Policies on Clusters and Namespaces

In [ ]:
./061-cluster-access-policies-import.sh

#### ES-62: Import Policy Templates

In [ ]:
./062-base-policy-templates-import.sh

#### ES-63: Import Policy Assignments

* Import policy assignments on organizations, clustergroups and workspaces

In [ ]:
./063-base-policy-assignments-import.sh

* Import policy assignments on clusters

In [ ]:
./063-cluster-policy-assignments-import.sh

### Import data protection resources

In [ ]:
./064-cluster-data_protection-import.sh